In [8]:
import os,json,pandas as pd,traceback
from dotenv import load_dotenv

In [12]:
load_dotenv()
KEY=os.getenv("OPEN_API_KEY")


In [15]:
from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(openai_api_key=KEY,name="gpt-3.5-turbo",temperature=0.2)



In [16]:
llm

ChatOpenAI(name='gpt-3.5-turbo', client=<openai.resources.chat.completions.Completions object at 0x000001B422341310>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001B42234E190>, temperature=0.2, openai_api_key='sk-proj-wdrDWLf8ZOIkREwgrkPHT3BlbkFJzzvYv5YcLe6SUMXATulL', openai_proxy='')

In [18]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import  get_openai_callback
import PyPDF2


In [19]:
Response_json={
    "1":{
        "mcq":"multiple choice questions",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct":"correct answer"

    }
    ,
        "2":{
        "mcq":"multiple choice questions",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct":"correct answer"
    },
        "3":{
        "mcq":"multiple choice questions",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct":"correct answer"

    }
    ,
        "4":{
        "mcq":"multiple choice questions",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct":"correct answer"
    }
}

In [32]:
Template="""Text: {text} \
you are an expert MCQ maker. given the above text, it is your job to create a quiz of \
    {number} multiple choice questions for {subject} students in {tone} tone.\
        make sure the questions are not repeated and check all the questions to be confirming the text\
            as well. make sure to format your response like Response_json below and use it as a guide.\
                 ensure  to make {number} mcqs \ 
###Response_json \
{response_json} """

In [33]:
quiz_generation_template=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=Template
)

In [34]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_template,output_key="quiz",verbose=True)

In [35]:
Template2=""" 
you are an expert english grammerian and writer. Given a multiple choice questions for {subject} students.\
you need to evaluate the complexity of the question and give a complete analysis of the quiz. only use at \
max 50 words for complexity. if the quiz is not as per the cognitive and analytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the \
students ability.
quiz_mcqs:
{quiz}

Check from an expert english writer of the above quiz:
"""

In [36]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject","quiz"],template=Template2)

In [37]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [39]:
generate_evaluation_chain=SequentialChain(chains=[quiz_chain,review_chain],
                                          input_variables=["text","number","subject","tone","response_json"],
                                          output_variables=["quiz","review"],
                                          verbose=True)

In [48]:
with open(r'C:\Users\User\mcqgen\data.txt','r') as f:
    TEXT=f.read()

In [46]:
json.dumps(Response_json)

'{"1": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [49]:
number=5
subject="Data science"
tone='simple'

In [50]:
with get_openai_callback() as cb:
    response=generate_evaluation_chain(
        {
            "text":TEXT,
            "number":number,
            "subject":subject,
            "tone":tone,
            "response_json":json.dumps(Response_json)
            }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Text: The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used 

In [55]:
print(cb)
print("-----"*50)
print(cb.total_tokens,cb.prompt_tokens,cb.completion_tokens,cb.total_cost)

Tokens Used: 3517
	Prompt Tokens: 3119
	Completion Tokens: 398
Successful Requests: 2
Total Cost (USD): $0.0054745
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
3517 3119 398 0.0054745


In [56]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive system

In [68]:
json_text=response.get("quiz")

In [72]:
response_json_result=json.loads(json_text.replace("###Response_json \n",''))

In [76]:
import pandas as pd
quiz_df=pd.DataFrame(response_json_result).T

In [78]:
quiz_df.to_csv("DataScience_Quiz.csv",index=False)